In [2]:
from bs4 import BeautifulSoup
import requests
import re
import time
import pandas as pd

In [3]:
def get_income_data_urls ():
    base = 'https://www.payscale.com/college-salary-report/bachelors'
    urls = []
    urls.append(base)
    for page_num in range(2, 63):
        urls.append('{}/page/{}'.format(base, page_num))
    return urls

In [4]:
def get_row_data(row):
    school_name_test = row.find('td', class_ = 'data-table__cell csr-col--school-name').find('a')
    #Some of the school names don't have a link to the website.
    if school_name_test == None:
        school_name = row.find('td', class_ = 'data-table__cell csr-col--school-name').find('img').text
    else:
        school_name = school_name_test.text
    early_career_pay = (row.find(text=re.compile('Early Career Pay'))
                        .parent.next_sibling.text
                        .replace('$', '').replace(',', ''))
    mid_career_pay = (row.find(text=re.compile('Mid-Career Pay'))
                        .parent.next_sibling.text
                        .replace('$', '').replace(',', ''))
    percent_stem = (row.find(text=re.compile('% STEM Degrees'))
                        .parent.next_sibling.text
                        .replace('%', ''))
    
    return [school_name, early_career_pay, mid_career_pay, percent_stem]

In [7]:
def get_table_all_pages(urls):
    income_data = []
    for url in urls:
        req = requests.get(url)
        soup = BeautifulSoup(req.content, 'html.parser')
        rows = soup.find_all('tr', class_ = 'data-table__row')
        for row in rows:
            income_data.append(get_row_data(row))
        time.sleep(.3)
    return income_data
        

In [9]:
def make_income_df():
    income_urls = get_income_data_urls()
    income_data = get_table_all_pages(income_urls)
    income_header = ['school', 'early_pay', 'mid_pay', '%stem']
    income_df = pd.DataFrame(income_data, columns = income_header)
    return income_df

In [10]:
income_df = make_income_df()

In [12]:
income_df[income_df['school'] == 'Alabama A & M University']

,school,early_pay,mid_pay,%stem
916,Alabama A & M University,48700,83500,20


In [13]:
len(income_df)

1550